# **Task 2**

In [23]:
#importing needed libraries
import numpy as np
import pandas as pd


In [24]:
#mounting google drive to access files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
#saving all csv files as matrixes
Schema = pd.read_csv('/content/drive/MyDrive/cs412/HW2/schema.csv')
ConversionRates = pd.read_csv('/content/drive/MyDrive/cs412/HW2/conversionRates.csv')
MultipleChoiceResponses = pd.read_csv('/content/drive/MyDrive/cs412/HW2/multipleChoiceResponses.csv',encoding='latin-1')
FreeFormResponses = pd.read_csv('/content/drive/MyDrive/cs412/HW2/freeformResponses.csv')

<ipython-input-25-4486a4d19605>:4: DtypeWarning: Columns (31,83,86,87,98,99,109,116,123,124,127,129,130,164) have mixed types. Specify dtype option on import or set low_memory=False.
  MultipleChoiceResponses = pd.read_csv('/content/drive/MyDrive/cs412/HW2/multipleChoiceResponses.csv',encoding='latin-1')
<ipython-input-25-4486a4d19605>:5: DtypeWarning: Columns (5,17,21,38,50) have mixed types. Specify dtype option on import or set low_memory=False.
  FreeFormResponses = pd.read_csv('/content/drive/MyDrive/cs412/HW2/freeformResponses.csv')


2a) What is the probability that a respondent is currently employed as a Programmer given they use C/C++ at work?


In [26]:
# Number of total respondents
TotalRespondents = MultipleChoiceResponses.shape[0]

# Calculate the probability of using C/C++ at work
TotalMultipleCplus = MultipleChoiceResponses[MultipleChoiceResponses['WorkToolsSelect'].str.contains("C/C+")==True].shape[0]
TotalFreeCplus = FreeFormResponses[(FreeFormResponses['WorkToolsFreeForm1'].str.contains("C/C+")==True)|
                                   (FreeFormResponses['WorkToolsFreeForm2'].str.contains("C/C+")==True)|
                                   (FreeFormResponses['WorkToolsFreeForm3'].str.contains("C/C+")==True)].shape[0]
TotalCPlusUsers = TotalMultipleCplus + TotalFreeCplus
ProbabilityOfCPlus = TotalCPlusUsers / TotalRespondents

# Calculate the probability of being a programmer given usage of C/C++ at work
ProgrammersUseCPlus = ((MultipleChoiceResponses[(MultipleChoiceResponses['WorkToolsSelect'].str.contains("C/C+")==True) &
                                                (MultipleChoiceResponses['CurrentJobTitleSelect'] == 'Programmer')].shape[0]) +
                       (FreeFormResponses[((FreeFormResponses['WorkToolsFreeForm1'].str.contains("C/C+")==True)|
                                           (FreeFormResponses['WorkToolsFreeForm2'].str.contains("C/C+")==True)|
                                           (FreeFormResponses['WorkToolsFreeForm3'].str.contains("C/C+")==True))&
                                           (FreeFormResponses['CurrentJobTitleFreeForm'].str.contains('Programmer') |
                                            FreeFormResponses['CurrentJobTitleFreeForm'].str.contains('programmer'))].shape[0]))
ProbProgrammerAndUsingCPlus = ProgrammersUseCPlus / TotalRespondents

# Print the result
print("Probability of being a programmer given usage of C/C++ at work:", ProbProgrammerAndUsingCPlus/ProbabilityOfCPlus)

Probability of being a programmer given usage of C/C++ at work: 0.02223675604970569


2b) What is the probability that a respondent is a Data Scientist given they have majored in computer science, mathematics or statistics?

In [27]:

# Calculate the probability of majoring in CS, Math, or Stats
MultiMajoredCsMathStats = MultipleChoiceResponses[(MultipleChoiceResponses['MajorSelect'] == 'Computer Science') |
                                                  (MultipleChoiceResponses['MajorSelect'] == 'Mathematics or Statistics')]

FreeMajoredCsMathStats = FreeFormResponses[(FreeFormResponses['MajorFreeForm'].str.contains("Computer Sciences")==True) |
                                           (FreeFormResponses['MajorFreeForm'].str.contains("computer sciences")==True) |
                                            (FreeFormResponses['MajorFreeForm'].str.contains("Mathematics")==True) |
                                            (FreeFormResponses['MajorFreeForm'].str.contains("mathematics")==True)|
                                            (FreeFormResponses['MajorFreeForm'].str.contains("statistics")==True)|
                                            (FreeFormResponses['MajorFreeForm'].str.contains("Statistics")==True)]

MajoredCsMathStats = FreeMajoredCsMathStats.shape[0] + MultiMajoredCsMathStats.shape[0]
ProbabilityOfCsMathStats = MajoredCsMathStats / TotalRespondents

# Calculate the probability of being a Data Scientist given major in CS, Math, or Stats
MultiDataScientistsCsMathStats = MultiMajoredCsMathStats[MultiMajoredCsMathStats['CurrentJobTitleSelect'] == 'Data Scientist']
FreeDataScientistsCsMatStats = FreeMajoredCsMathStats[(FreeMajoredCsMathStats['CurrentJobTitleFreeForm'] == 'Data Scientist') |
                                                      (FreeMajoredCsMathStats['CurrentJobTitleFreeForm'] == 'data scientist') ]
DataScientistsCsMathStats = MultiDataScientistsCsMathStats.shape[0] + FreeDataScientistsCsMatStats.shape[0]
ProbabilityOfDataScientistGivenCsMathStats = DataScientistsCsMathStats / MajoredCsMathStats

# Print the result
print("Probability of being a Data Scientist given major in CS, Math, or Stats:", ProbabilityOfDataScientistGivenCsMathStats/ProbabilityOfCsMathStats)


Probability of being a Data Scientist given major in CS, Math, or Stats: 0.45042344067953877


2c) What is the probability that a respondent works in the Technology industry given that they earn more than 40,000 USD annually?

In [28]:
# Clean data from non-int values
MultipleChoiceResponses['CompensationAmount'] = pd.to_numeric(MultipleChoiceResponses['CompensationAmount'], errors='coerce')

# Merge multiple choice responses with conversion rates based on Currency column
Merged = pd.merge(MultipleChoiceResponses, ConversionRates, left_on='CompensationCurrency', right_on='originCountry')


# Convert CompensationAmount to USD using conversion rates
Merged['CompensationAmountUSD'] = Merged['CompensationAmount'].astype(float) * Merged['exchangeRate']
MultipleChoiceResponses['CompensationAmountUSD'] = Merged['CompensationAmountUSD']

# Calculate the probability of having compensation more than 40000 USD annualy
ProbabilityOfComp40000USD = Merged[Merged['CompensationAmountUSD']>= 40000].shape[0]/TotalRespondents


# Calculate the probability of working in Technology industry and havin more than 40000 USD annualy
ProbabilityOfIndustry40000USD = MultipleChoiceResponses[(MultipleChoiceResponses['CompensationAmountUSD'] >= 40000) &
                                                        (MultipleChoiceResponses['EmployerIndustry'] == "Technology")].shape[0]/TotalRespondents

print("Probability of a respondent works in the Technology industry given that they earn more than 40,000 USD annually:", ProbabilityOfIndustry40000USD/ProbabilityOfComp40000USD)


Probability of a respondent works in the Technology industry given that they earn more than 40,000 USD annually: 0.14406402845709207


2d) What is the joint probability of a respondent being over 30 years old and having a at least a Bachelor’s degree?


In [ ]:

# Count the number of respondents over 30 years old
Prob30 = MultipleChoiceResponses[(MultipleChoiceResponses['Age'] > 30)].shape[0]/TotalRespondents
# Count the number of respondents with at least a Bachelor's degree
ProbDegree = MultipleChoiceResponses[((MultipleChoiceResponses['FormalEducation'] == "Bachelor's degree") |
                                    (MultipleChoiceResponses['FormalEducation'] == "Master's degree") |
                                    (MultipleChoiceResponses['FormalEducation'] == "Professional degree") |
                                    (MultipleChoiceResponses['FormalEducation'] == "Doctoral degree") )].shape[0]/TotalRespondents

# Calculate the joint probability
JointProbability = Prob30 * ProbDegree

print("Joint probability of being over 30 years old and having at least a Bachelor's degree:", JointProbability)

Joint probability of being over 30 years old and having at least a Bachelor's degree: 0.3787156348865649


2e) What is the probability that a respondent is a Data Scientist who majored in Computer
Science, Mathematics or statistics?


In [ ]:
# Calculate number of respondets that Data Scientist who majored in Computer Science, Mathematics or statistics
Multi = MultipleChoiceResponses[(MultipleChoiceResponses['MajorSelect'].isin(['Computer Science', 'Mathematics or Statistics'])) &
                                (MultipleChoiceResponses['CurrentJobTitleSelect'] == 'Data Scientist')].shape[0]
Free = FreeFormResponses[((FreeFormResponses['MajorFreeForm'].str.contains("Computer Sciences")==True) |
                          (FreeFormResponses['MajorFreeForm'].str.contains("computer sciences")==True) |
                          (FreeFormResponses['MajorFreeForm'].str.contains("Mathematics")==True) |
                          (FreeFormResponses['MajorFreeForm'].str.contains("mathematics")==True)|
                          (FreeFormResponses['MajorFreeForm'].str.contains("statistics")==True)|
                          (FreeFormResponses['MajorFreeForm'].str.contains("Statistics")==True)) &
                         ((FreeFormResponses['CurrentJobTitleFreeForm'].str.contains("Data Scientist")==True)|
                          (FreeFormResponses['CurrentJobTitleFreeForm'].str.contains("data scientist")==True))].shape[0]


# Calculate the probability of being a Data Scientist and major in CS, Math, or Stats
DataScientistsCMS = (Multi + Free)/TotalRespondents


print("Joint probability of being a Data Scientist who majored in Computer Science, Mathematics or statistics:", DataScientistsCMS)

Joint probability of being a Data Scientist who majored in Computer Science, Mathematics or statistics: 0.03170614979660206


2f) What is the joint probability that a respondent is from France, earns less than 100,000
USD annually, and uses Cross-Validation Often or Most of the time?


In [ ]:
ProbFrance = MultipleChoiceResponses[(MultipleChoiceResponses['Country'] == 'France')].shape[0]/TotalRespondents
ProbErningLessGivenFrance = (MultipleChoiceResponses[(MultipleChoiceResponses['Country'] == 'France')&
                                                    (MultipleChoiceResponses['CompensationAmountUSD'] < 100000)].shape[0])/ProbFrance

JointProbability2 = MultipleChoiceResponses[(MultipleChoiceResponses['Country'] == 'France') &
                                            (MultipleChoiceResponses['CompensationAmountUSD'] < 100000) &
                                            (MultipleChoiceResponses['WorkMethodsFrequencyCross-Validation'].isin(['Often', 'Most of the time']))].shape[0]/TotalRespondents

print("Joint probability of coming from France, earning less than 100,000 USD annually, and using Cross-Validation Often or Most of the time?:", JointProbability2)

Joint probability of coming from France, earning less than 100,000 USD annually, and using Cross-Validation Often or Most of the time?: 0.002512562814070352



2g) What is the probability that a respondent uses C/C++ at work given that they are employed
as a Programmer?

In [ ]:
Free = FreeFormResponses[(FreeFormResponses['CurrentJobTitleFreeForm'].str.contains("programmer")==True) |
                         (FreeFormResponses['CurrentJobTitleFreeForm'].str.contains("Programmer")==True)].shape[0]
Multi = MultipleChoiceResponses[MultipleChoiceResponses['CurrentJobTitleSelect'] == 'Programmer'].shape[0]

# Calculate probability of being programmer
TotalProgrammers = Free + Multi
ProbProgrammer = TotalProgrammers / TotalRespondents

ProbabilityOfCPlusGivenProgrammer = ProbProgrammerAndUsingCPlus/ProbProgrammer

print("Probability of a respondent uses C/C++ at work given that they are employed as a Programmer:", ProbabilityOfCPlusGivenProgrammer)

Probability of a respondent uses C/C++ at work given that they are employed as a Programmer: 0.07327586206896552


2h) Given the probability of a respondent wearing glasses is 0.15, and the probability of a
respondent wearing glasses given they have a PhD is 0.25, find the probability of a respondent having a PhD
given that they wear glasses.

In [ ]:
# Given probabilities
ProbGlass = 0.15  # Probability of wearing glasses
ProbGlassGivenPHD = 0.25  # Probability of wearing glasses given having a PhD

# Calculate the probability of having a PHD
ProbPHD = MultipleChoiceResponses[MultipleChoiceResponses['FormalEducation'] == 'Doctoral degree'].shape[0]/TotalRespondents

# Calculate the probability of having a PhD given wearing glasses using Bayes' theorem
ProbPHDGivenGlasses = (ProbGlassGivenPHD*ProbPHD)/ProbGlass

print("Probability of having a PhD given wearing glasses:", ProbPHDGivenGlasses)

Probability of having a PhD given wearing glasses: 0.23400733827869508
